# Query Checking in Declare4Py

This tutorial explains how to perform the query checking of a DECLARE constraint in a log and how to browse the results.

After importing the Declare4Py package, a `Declare4Py` object has to be instantiated to load the log.

In [3]:
import sys
import os
import pathlib

SCRIPT_DIR = pathlib.Path("..", "src").resolve()
sys.path.append(os.path.dirname(SCRIPT_DIR))

from src.declare4py.ProcessModels.DeclareModel import DeclareModel
from src.declare4py.ProcessMiningTasks.DeclareQueryChecking.QueryChecker import QueryChecker
from src.declare4py.ProcessMiningTasks.DeclareQueryChecking.ResultsBrowser import ResultsBrowser
from src.declare4py.D4PyEventLog import D4PyEventLog

log_path = os.path.join("..", "tests", "Sepsis Cases.xes.gz")
event_log = D4PyEventLog()
event_log.parse_xes_log(log_path)

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

The query checking is performed with the `query_checking` function. This takes as input the boolean parameter `consider_vacuity=true` that means that vacuously satisfied traces are considered as satisfied, violated otherwise. The parameter `template_str` sets the DECLARE constraint. Then the string parameters `activation` and `target` sets the variables to ask. If these parameters are not set then they are considered as variables to which compute the assignments. The input string parameters `act_cond`, `trg_cond`, `time_cond` are used to set the activation, target and time conditions (if needed), respectively. The float `min_support` parameter sets the support to be satisfied in the log by the variable assignments.

Two related query checking tasks can be performed by setting the `return_first` boolean parameter:
1. `return_first=False` returns all the variables assignments that satisfy the support in the log.
2. `return_first=True` returns only one variables assignment (the first) that satisfy the support in the log. This saves computational time when one is only interested in the existence of a variable assignment with a given support.

The returned data structure is a Python dictionary with keys the Declare constraints satisfying the assignments. The values are a structured representations of these constraints. Let's compute the assignments for the target by setting `template_str='Chain Response'`, `activation='IV Antibiotics'`, `act_cond='A.org:group is A'` and `min_support=0.2`:

In [4]:
query_checker = QueryChecker(log=event_log, consider_vacuity=False, template='Chain Response', activation='IV Antibiotics', activation_condition='A.org:group is A', min_support=0.2, activity_attribute="concept:name", return_first=False)
query_check_res: ResultsBrowser = query_checker.run()

The query checking results can be filtered with the `filter_query_checking()` function whose parameter `queries` takes as input a list of variables to ask. In the above example, we set the template and activation and ask for the targets. We now filter the results by returning only the targets:

In [5]:
query_check_res.filter_query_checking(queries=['target'])

,target
0,Admission NC


In the following example we compute the assignments both for the activation and the target by setting `template_str='Response'` and `min_support=0.8`:

In [8]:
query_checker = QueryChecker(log=event_log, consider_vacuity=False, template='Response', min_support=0.8, activity_attribute="concept:name", return_first=False)
query_check_res: ResultsBrowser = query_checker.run()

In this example, we set the template and ask for the activations and targets. We now filter the results by returning both activations and targets:

In [11]:
query_check_res.filter_query_checking(queries=['activation', 'target', 'template'])

,activation,target,template
0,ER Triage,Leucocytes,Response
1,ER Triage,ER Sepsis Triage,Response
2,ER Triage,CRP,Response
3,ER Registration,ER Triage,Response
4,ER Registration,Leucocytes,Response
5,ER Registration,LacticAcid,Response
6,ER Registration,ER Sepsis Triage,Response
7,ER Registration,CRP,Response
8,ER Sepsis Triage,Leucocytes,Response
9,ER Sepsis Triage,CRP,Response


In [10]:
query_checker = QueryChecker(log=event_log, consider_vacuity=False, activation='ER Registration', target='CRP', min_support=0.2, activity_attribute="concept:name", return_first=False)
query_check_res: ResultsBrowser = query_checker.run()
    
query_check_res.filter_query_checking(queries=['activation', 'target'])

AttributeError: 'TemplateConstraintChecker' object has no attribute 'mpSuccession'